In [1]:
import pandas as pd
import numpy as np
from scipy import io
import os, errno

In [2]:
# consolidate the true trials tDetect and exploration time across ALL the data.

direc1 = '/Users/Mahad/Documents/GitHub/ManducaMultiSenseFeeding/Data_From_Step_3'
files = [file for file in os.listdir(direc1) if file.endswith('.csv')]


# load all the files with IRdtect data
direc2 = '/Users/Mahad/Documents/GitHub/ManducaMultiSenseFeeding/Data_From_Step_4'
IR_data_files = [file for file in os.listdir(direc2)]

In [3]:
new_path = '/Users/Mahad/Documents/GitHub/ManducaMultiSenseFeeding/Data_From_Step_5/'

try:
    os.makedirs(new_path)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

for fname in files:
    df = pd.read_csv(direc1 + '/' + fname)
    # pull out all the trueTrials and reset the index to default to get true trial #
    trueTrials = df[df.label == 1].loc[:, ['In_Frame', 'Out_Frame']].reset_index(drop = True)
    
    # remove "cropped" from filename to match IR file
    if fname.find('_cut') :
        new_fname = fname.replace('_cut','')
    else: 
        new_fname = fname
    
    # load IR files and get indices of all the frames where IRDetect == 1
    IR_detect_file = [file for file in IR_data_files if new_fname[:-22] in file]
    print(IR_detect_file[0])
    loadedIRdetect = np.load(direc2 + '/' + IR_detect_file[0])
    indx = np.where(loadedIRdetect)
    indx = indx[0].reshape(-1)
    
    # for each true trial, find the first frame where IRDetect == True
    proboscisDetect = []
    discoveryTime = []
    for framein, frameout in zip(trueTrials.In_Frame, trueTrials.Out_Frame):
        allframes = [frame for frame in indx if framein < frame < frameout]
        if allframes == []:
            detect = np.nan
        else:
            detect = min(allframes)
        Time  = detect - framein
        proboscisDetect.append(detect)
        discoveryTime.append(Time)

    df1 = pd.DataFrame({'ProboscisDetect': proboscisDetect})
    df2 = pd.DataFrame({'DiscoveryTime' : discoveryTime})

    new_data = pd.concat([trueTrials, df1, df2], axis = 1)
    new_data.to_csv(new_path + fname[:-22] + '_RawDataForExplorationTime.csv')

L0.1_c-3_m12.mat_IRdetect.npy
L0.1_c-3_m20.mat_IRdetect.npy
L0.1_c-3_m21.mat_IRdetect.npy
L0.1_c-3_m22.mat_IRdetect.npy
L0.1_c-3_m23.mat_IRdetect.npy
L0.1_c-3_m24.mat_IRdetect.npy
L0.1_c-3_m25.mat_IRdetect.npy
L0.1_c-3_m2.mat_IRdetect.npy
L0.1_c-3_m8.mat_IRdetect.npy
L50_c-3_m10.mat_IRdetect.npy
L50_c-3_m12.mat_IRdetect.npy
L50_c-3_m13.mat_IRdetect.npy
L50_c-3_m14.mat_IRdetect.npy
L50_c-3_m15.mat_IRdetect.npy
L50_c-3_m21.mat_IRdetect.npy
L50_c-3_m22.mat_IRdetect.npy
L50_c-3_m24.mat_IRdetect.npy
L50_c-3_m2.mat_IRdetect.npy
L50_c-3_m6.mat_IRdetect.npy


In [4]:
fname

'L50_c-3_m6_MothInOut_labeled.csv'